In [170]:
from anateldb.reading import *
from fastcore.xtras import Path
LATITUDE = (-33.7017531, 5.2666664)  # Arroio Chuy RS, Monte Caburaí RR
LONGITUDE = (-75.3709938, -32.423786)
FOLDER = Path.cwd().parent / 'dados'

In [171]:
base = read_base(FOLDER)

In [172]:
LAT = ~base.Latitude.between(*LATITUDE)
LON = ~base.Longitude.between(*LONGITUDE)
base[LAT | LON].shape

(0, 18)

In [128]:
wrong_lat = base[LAT]
wrong_long = base[LON]
both = base.loc[LAT & LON]
both.shape

(701, 17)

In [129]:
wrong_long = wrong_long[~wrong_long.index.isin(both.index)]
wrong_long.shape

(259, 17)

In [130]:
wrong_lat = wrong_lat[~wrong_lat.index.isin(both.index)]
wrong_lat.shape

(75, 17)

In [148]:
import folium
from folium import plugins
mapa = folium.Map(location=[-15.7797200, -47.9297200], zoom_start=4)
ambas = folium.FeatureGroup("Ambas")
latitude = folium.FeatureGroup("Latitude")
longitude = folium.FeatureGroup("Longitude")

In [149]:
from tqdm.auto import tqdm
for row in tqdm(both.itertuples(), total=both.shape[0]):
    folium.CircleMarker((row.Latitude, row.Longitude),
                        color="#000",
                        fill_color="red",
                        fill_opacity=1,
                        radius= 3, 
                        weight=1,
                        opacity=2,
                        tooltip=f'{row.Frequência} MHz <br>Município/UF: {row.Município}/{row.UF}',
                        popup=folium.map.Popup(both.loc[row.Index].to_frame().T.to_html(index=False))                    
                    ).add_to(ambas)

  0%|          | 0/701 [00:00<?, ?it/s]

In [150]:
from tqdm.auto import tqdm
for row in tqdm(wrong_lat.itertuples(), total=wrong_lat.shape[0]):
    folium.CircleMarker((row.Latitude, row.Longitude),
                        color="#000",
                        fill_color="yellow",
                        fill_opacity=1,
                        radius= 3, 
                        weight=1,
                        opacity=2,
                        tooltip=f'{row.Frequência} MHz <br>Município/UF: {row.Município}/{row.UF}',
                        popup=folium.map.Popup(wrong_lat.loc[row.Index].to_frame().T.to_html(index=False))                    
                    ).add_to(latitude)

  0%|          | 0/75 [00:00<?, ?it/s]

In [151]:
from tqdm.auto import tqdm
for row in tqdm(wrong_long.itertuples(), total=wrong_long.shape[0]):
    folium.CircleMarker((row.Latitude, row.Longitude),
                        color="#000",
                        fill_color="cyan",
                        fill_opacity=1,
                        radius= 3, 
                        weight=1,
                        opacity=2,
                        tooltip=f'{row.Frequência} MHz <br>Município/UF: {row.Município}/{row.UF}',
                        popup=folium.map.Popup(wrong_long.loc[row.Index].to_frame().T.to_html(index=False))                    
                    ).add_to(longitude)

  0%|          | 0/259 [00:00<?, ?it/s]

In [152]:
ambas.add_to(mapa)
latitude.add_to(mapa)
longitude.add_to(mapa)

In [153]:
folium.LayerControl(collapsed=False).add_to(mapa)
mapa

In [77]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
geocoder = Nominatim(user_agent="anateldb")
geocode = RateLimiter(geocoder.geocode, min_delay_seconds = 1, return_value_on_exception = None) 

In [137]:
base['Coordenadas_do_Município'] = False

In [169]:
from time import sleep
for row in tqdm(base[LAT | LON].itertuples(), total=base[LAT | LON].shape[0]):
    location = geocoder.geocode(f'{row.Município}, {row.UF}, Brasil')
    print(location.raw)
    base.loc[row.Index, ['Latitude', 'Longitude']] = location.latitude, location.longitude
    base.loc[row.Index, 'Coordenadas_do_Município'] = True
    sleep(1)    

  0%|          | 0/38 [00:00<?, ?it/s]

{'place_id': 169128647, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 255739480, 'boundingbox': ['-3.9166033', '-3.7925131', '-32.4852501', '-32.3504099'], 'lat': '-3.8545643', 'lon': '-32.37861623300567', 'display_name': 'Parque Nacional Marinho de Fernando de Noronha, Fernando de Noronha, Região Geográfica Imediata do Recife, Região Geográfica Intermediária do Recife, Pernambuco, Região Nordeste, 53990-000, Brasil', 'class': 'boundary', 'type': 'protected_area', 'importance': 1.009101312224902}
{'place_id': 169128647, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 255739480, 'boundingbox': ['-3.9166033', '-3.7925131', '-32.4852501', '-32.3504099'], 'lat': '-3.8545643', 'lon': '-32.37861623300567', 'display_name': 'Parque Nacional Marinho de Fernando de Noronha, Fernando de Noronha, Região Geográfica Imediata do Recife, Região Geográfica Intermediária 

In [165]:
base.to_parquet(FOLDER / 'base.parquet.gzip', compression='gzip')

In [155]:
mapa.save('Verificação_de_Coordenadas.html')	